<a href="https://colab.research.google.com/github/koh-hongQ/KMS_NIMS_AI_Camp/blob/main/1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dataset 과 Dataloader

In [ ]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

MNIST_training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
    )

MNIST_test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [ ]:
x, y = MNIST_training_data[0]
print(x.shape)
print(x.squeeze().shape)
plt.imshow(x.squeeze(), cmap = 'gray')
plt.show()
print(y)

In [ ]:
MNIST_training_data[0][1]

In [ ]:
len(MNIST_training_data)

In [ ]:
MNIST_training_data.classes

In [ ]:
from torch.utils.data import random_split
print(len(MNIST_training_data))
dataset_train_split, dataset_val_split = random_split(MNIST_training_data, [50000, 10000])
print(len(dataset_train_split), len(dataset_val_split))

커스텀 Dataset

In [ ]:
from torchvision.datasets.mnist import read_sn3_pascalvincent_tensor
class Custom_MNIST(Dataset):
    def __init__(self, path_data, path_label):
        self.data = read_sn3_pascalvincent_tensor(path_data)
        self.label = read_sn3_pascalvincent_tensor(path_label)
    def __len__(self):
        return self.data.shape[0]
    def __getitem__(self, idx):
        return self.data[idx].float(), self.label[idx]
import os
path_data = os.path.join('data', 'MNIST', 'raw', 't10k-images-idx3-ubyte')
path_label = os.path.join('data', 'MNIST', 'raw', 't10k-labels-idx1-ubyte')
print(path_data)
print(path_label)
custom_dataset = Custom_MNIST(path_data, path_label)

In [ ]:
import os
path_data = os.path.join('data', 'MNIST', 'raw', 't10k-images-idx3-ubyte')
path_label = os.path.join('data', 'MNIST', 'raw', 't10k-labels-idx1-ubyte')
print(path_data)
print(path_label)

In [ ]:
from torchvision.datasets.mnist import read_sn3_pascalvincent_tensor
data = read_sn3_pascalvincent_tensor(path_data)
print(data.shape, type(data))

In [ ]:
label = read_sn3_pascalvincent_tensor(path_label)
print(label.shape, type(label))

In [ ]:
custom_dataset.data.shape, custom_dataset.label.shape

In [ ]:
path_data = os.path.join('data', 'MNIST', 'raw', 't10k-images-idx3-ubyte')
path_label = os.path.join('data', 'MNIST', 'raw', 't10k-labels-idx1-ubyte')
custom_dataset = Custom_MNIST(path_data, path_label)
print(len(custom_dataset))
x, y = custom_dataset[0]
print(x.shape)
plt.imshow(x, cmap = 'gray')
plt.show()
print(y)

연습문제: data의 개수 n_samples를 argument로 하는 Dataset class를 생성하라. x_1, x_2는 [0, 1] uniform 하게 분포하고 y는 2x_1 + x_2 + 3.5 + noise로 생성된다. noise는 평균이 0, 표준편차 0.1인 gaussian 분포를 따른다.

Dataloader

In [ ]:
from torch.utils.data import DataLoader
dataloader = DataLoader(MNIST_training_data, batch_size=32, shuffle=True)
print(dataloader.dataset)
print(len(dataloader))

In [ ]:
for x, y in dataloader:
    print(x.shape, y.shape)
    plt.imshow(x[0].squeeze(), cmap = 'gray')
    plt.show()
    break

In [ ]:
x,y = next(iter(dataloader))
print(x.shape, y.shape)
plt.imshow(x[0].squeeze(), cmap = 'gray')
plt.show()

Neural Network Module

In [ ]:
from torch import nn
import torch
linear = nn.Linear(2,1)
for param in linear.parameters():
    print(param)


In [ ]:
class Affine(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(2,1)
    def forward(self, x):
        return self.linear(x)
model = Affine()
for param in linear.parameters():
    print(param)

In [ ]:
x = torch.randn(64,2)
print(model(x).shape)

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(device)
model.to(device)
x = x.to(device)
y = y.to(device)
print(x,y)

In [ ]:
from torch.utils.data import DataLoader
dataloader = DataLoader(regression_dataset, batch_size=10, shuffle=True)
x,y = next(iter(dataloader))
x = x.to(device)
y = y.to(device)
y_hat = model(x)
print(y_hat.shape, y.shape)

In [ ]:
from torch.nn import MSELoss
loss_function = MSELoss()
loss = loss_function(y_hat, y)

In [ ]:
for param in model.parameters():
    print(param.grad)
loss.backward()
for param in model.parameters():
    print(param.grad)

In [ ]:
for param in model.parameters():
    print(param)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
optimizer.step()
print('Processing SGD')
for param in model.parameters():
    print(param)

In [ ]:
from torch.utils.data import Dataset, DataLoader
class LinearRegressionDataset(Dataset):
    def __init__(self, n_samples):
        self.x = torch.rand(n_samples, 2)                                                                 # 입력 x
        noise = torch.randn(n_samples) * 0.1                                                              # 노이즈
        self.y = (2 * self.x[:, 0] + self.x[:, 1] + 3.50 + noise).unsqueeze(dim=1)                        # 출력 y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

model = Affine()
regression_dataset = LinearRegressionDataset(1000)
dataloader = DataLoader(regression_dataset, batch_size=10, shuffle=True)
loss_function = MSELoss()
learning_rate = 1e-3
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

def train_epoch(model, dataloader, loss_function, optimizer):
    model.train()
    model.to(device)
    for x,y in dataloader:
        x = x.to(device)
        y = y.to(device)
        y_hat = model(x)
        loss = loss_function(y_hat, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    if (epoch+1)%10 == 0:
        print(f'{epoch+1}th epoch')
        for param in model.parameters():
            print(param.data)
    return loss


In [ ]:
for epoch in range(100):
    loss = train_epoch(model, dataloader, loss_function, optimizer)

Multilayer Perceptron

In [ ]:
from torch import nn
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear1 = nn.Linear(28*28, 512)
        self.relu1 = nn.ReLU()
        self.linear2 = nn.Linear(512, 512)
        self.relu2 = nn.ReLU()
        self.linear3 = nn.Linear(512, 10)
        self.softmax = nn.Softmax(dim=1)
    def forward(self, x):
        x = self.flatten(x)
        x = self.linear1(x)
        x = self.relu1(x)
        x = self.linear2(x)
        x = self.relu2(x)
        x = self.linear3(x)
        x = self.softmax(x)
        return x
model1 = NeuralNetwork()
for param in model1.parameters():
    print(param.shape)

In [ ]:
class NeuralNetwork2(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
        self.softmax = nn.Softmax(dim=1)
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return self.softmax(logits)
model2 = NeuralNetwork2()
for param in model2.parameters():
    print(param.shape)

In [ ]:
from torch.utils.data import DataLoader
MNIST_train_dataloader = DataLoader(MNIST_training_data, batch_size=64, shuffle=True)
MNIST_test_dataloader = DataLoader(MNIST_test_data, batch_size=64, shuffle=False)
model2 = NeuralNetwork2()
optimizer = torch.optim.SGD(model2.parameters(), lr=1e-3)
loss_function = torch.nn.CrossEntropyLoss()

In [ ]:
x, y = next(iter(MNIST_train_dataloader))
print(x.shape), print(y.shape)
y_hat = model2(x)
print(y_hat.shape)
loss_function(y_hat, y)

In [ ]:
def train_epoch(model, dataloader, loss_function, optimizer):
    train_loss = 0
    num_batches = len(dataloader)
    model.train()
    if torch.cuda.is_available():
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')
    model.to(device)
    for x,y in dataloader:
        x = x.to(device)
        y = y.to(device)
        y_hat = model(x)
        loss = loss_function(y_hat, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        train_loss += loss.item()
    train_loss/= num_batches
    print(f"Train Error: Avg loss: {train_loss}")
    return train_loss

def test_epoch(model, dataloader, loss_function):
    model.eval()
    test_loss, correct = 0, 0
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    if torch.cuda.is_available():
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')
    model.to(device)
    with torch.no_grad():
        for x,y in dataloader:
            x = x.to(device)
            y = y.to(device)
            y_hat = model(x)
            loss = loss_function(y_hat, y)
            test_loss += loss.item()
            correct += (y_hat.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: Accuracy: {(100*correct)}%, Avg loss: {test_loss}")
    return test_loss, correct

train_loss_list = []
test_loss_list = []
for epoch in range(20):
    train_loss = train_epoch(model2, MNIST_train_dataloader, loss_function, optimizer)
    test_loss, correct = test_epoch(model2, MNIST_test_dataloader, loss_function)
    train_loss_list.append(train_loss)
    test_loss_list.append(test_loss)
plt.plot(range(len(train_loss_list)), train_loss_list)
plt.plot(range(len(test_loss_list)), test_loss_list)
plt.show()

Model 저장 및 불러오기

In [ ]:
torch.save(model2, 'model.pth')

In [ ]:
model = torch.load('model.pth', weights_only=False)

In [ ]:
torch.save(model2.state_dict(), 'model_weights.pth')

In [ ]:
for name, param in model2.state_dict().items():
    print(name, param.shape)

In [ ]:
model = NeuralNetwork2()
model.load_state_dict(torch.load('model_weights.pth'))
test_epoch(model2, MNIST_test_dataloader, loss_function)

Learning Rate Scheduling

In [ ]:
from torch.optim.lr_scheduler import ExponentialLR, ReduceLROnPlateau
scheduler = ReduceLROnPlateau(optimizer)
for epoch in range(10):
    loss = train_epoch(model2, MNIST_train_dataloader, loss_function, optimizer)
    scheduler.step(loss)

In [ ]:
scheduler.get_last_lr()

In [ ]:
model = nn.Dropout()
x = torch.randn(3)
model.train()
print('Training mode')
for i in range(5):
    print(model(x))
model.eval()
print('Evaluation mode')
for i in range(5):
    print(model(x))